In [1]:
from Crypto.Cipher import AES
import filetype
from Crypto.PublicKey import RSA
from glob import glob
from math import gcd
from sympy import *
import os

In [2]:
ruta = "./keys/"

In [3]:
aes = ruta + "pablo.vega.gallego_AES_pseudo.enc"

In [13]:
pem = ruta + "pablo.vega.gallego_pubkeyRSA_pseudo.pem"

pem = ruta + 'natasha.trojan_pubkeyRSA_pseudo.pem'

In [5]:
rsa = ruta + "pablo.vega.gallego_RSA_pseudo.enc"

In [14]:
with open(pem, 'r') as f:
    rsakey = f.read()

pubkey = RSA.importKey(rsakey)
personal = pubkey.n
public_exponent = pubkey.e
print(personal)
print()
print(public_exponent)

21228709888260993109738529818092331932017625154792132658524539262261695428472333282238868788988830233993262565457190379712573509325772228622985502106064673014779230362576916690807467149087625889102461212634961805480984957312450261548919984169493892133015536155758300914767745245258165563771508208144796530313571810409714310998922221766450889468049655436793779994891476780612207817517407008396701644614922727860794070071130084187111808701112953498899394441214461098124270194608340738237694954697622550811680350905977649299820008903967471297393232894320581670732540393517218730450741857927882197017278655790244328926189

65537


In [15]:
len(bin(personal)[2:]) % 4 == 0

True

In [16]:
N = len(bin(personal)[2:])//4
N

512

In [17]:
binpersona = bin(personal)[2:]

In [18]:
primero = binpersona[0:N]
segundo = binpersona[N:2*N]
tercero = binpersona[2*N:3*N]
cuarto = binpersona[3*N:4*N]

In [19]:
personal == int(primero+segundo+tercero+cuarto, 2)

True

In [20]:
var('x')
for i in range(0, 2**N):
    c = int(bin(i)[2:] + segundo + tercero, 2)
    primeroAct = bin(int(primero, 2) - i)[2:]
    RS = int(primeroAct + cuarto, 2)
    RSinv = int(cuarto + primeroAct, 2)
    r2s2 = c - RSinv
    rs2 = r2s2 + 2*RS
    rs = sqrt(rs2)
    sol = solve(x**2 - x*rs + RS, x)
    if type(sol[0]) == Integer:
        print(sol)
        break

[9374304029389676731118239817282212946710593812728057328804809275979400326783275909773320912557274933077812019697023993333812096630983187439627817023997145, 12597054478421417822621364716021607058230767954589934264498239771721149603200799594203091533950150551468880092278902908930834791730802831672326137656468277]


In [13]:
sol

[8120470032484937215252352830373054957005429959946418105737831966927293227804430640111052886726065514448123577935787451501083134212657907371004421738067647,
 11769487670416003154947777018232547194230957256023044363985701026726845357363799664299211884815687412739135955509843804465741245736272155354245040492439711]

In [14]:
p = int(bin(sol[0])[2:] + bin(sol[1])[2:], 2)
q = int(bin(sol[1])[2:] + bin(sol[0])[2:], 2)

In [15]:
p * q == personal

True

In [62]:
phi = (p-1)*(q-1)

private_exponent = pow(public_exponent, -1, phi)

key = RSA.construct((int(personal), int(public_exponent), int(private_exponent)), consistency_check=False)

In [63]:
f = open('privateKey_Pablo_Vega.pem','wb')
f.write(key.exportKey('PEM'))
f.close()

In [64]:
os.system('openssl rsautl -decrypt -in pablo.vega.gallego_RSA_pseudo.enc -out AESkey.txt -inkey privateKey_Pablo_Vega.pem')

The command rsautl was deprecated in version 3.0. Use 'pkeyutl' instead.


0

In [65]:
os.system('openssl enc -d -aes-128-cbc -pbkdf2 -kfile AESkey.txt -in pablo.vega.gallego_AES_pseudo.enc -out desencriptado.txt')

0

In [66]:
with open('desencriptado.txt', 'rb') as f:
    desencriptado = f.read()

In [67]:
filetype.guess(desencriptado)

In [68]:
with open('foto.png', "wb") as binary_file:
        binary_file.write(desencriptado)